<a href="https://colab.research.google.com/github/fahmidjobbi/TP-COLLAB/blob/main/GanMnistFashion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from numpy import zeros, ones, expand_dims, asarray
from numpy.random import randn, randint

from tensorflow.keras.optimizers import Adam
from keras.models import Model, load_model
from keras.layers import Input, Dense, Reshape, Flatten
from keras.layers import Conv2D, Conv2DTranspose, Concatenate
from keras.layers import LeakyReLU, Dropout, Embedding
from keras.layers import BatchNormalization, Activation
from keras import initializers
from keras.initializers import RandomNormal
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
import matplotlib.pyplot as plt 
import numpy as np
from math import sqrt

In [ ]:
from keras.datasets import fashion_mnist
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()
X_train1 = X_train.astype(np.float32) / 255
#X_train2 = np.expand_dims(X_train1, axis=3)#ajouter une dimansion a la fin
print(X_train1.shape)

In [ ]:
print(X_train[20])

In [ ]:
X_train.shape

In [ ]:
plt.imshow(X_train[10],'gray')

In [ ]:
plt.figure(figsize =(10, 10))
for i in range(25):
    plt.subplot(5, 5, i + 1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(X_train1[i], cmap = plt.cm.binary)
plt.show()

In [ ]:
#generate a matrix randomly
from numpy.random import randn, randint
def generate_latent_points(latent_dim, n_samples):
    x_input = randn(latent_dim * n_samples) #standard normal distribution vecteur of size latent_dim * n_samples
    z_input = x_input.reshape(n_samples, latent_dim) #convert vector to matrix
    return z_input

In [ ]:
Z_input=generate_latent_points(100,64)
print(Z_input)


In [ ]:
Z_input.shape

In [ ]:
plt.imshow(Z_input,'gray')

In [ ]:
def generate_real_samples(X_train, n_samples): #extract 64 images randomly from original real data
    ix = randint(0, X_train.shape[0], n_samples) 
    X = X_train[ix]  
    y = ones((n_samples, 1)) 
    return X, y

In [ ]:
Real_Data,y_real=generate_real_samples(X_train1,64)
Real_Data.shape
#print(y_real)

In [ ]:
y_real.shape
print(y_real)

In [ ]:
def generate_fake_samples(generator, latent_dim, n_samples):
    z_input = generate_latent_points(latent_dim, n_samples)
    images = generator.predict(z_input)  
    y = zeros((n_samples, 1))
    return images, y

In [ ]:
def define_generator(latent_dim): 
    init = RandomNormal(stddev=0.02)
    in_lat = Input(shape=(latent_dim,)) 
    gen = Dense(256, kernel_initializer=init)(in_lat)
    gen = LeakyReLU(alpha=0.2)(gen)
    gen = Dense(512, kernel_initializer=init)(gen)
    gen = LeakyReLU(alpha=0.2)(gen)
    gen = Dense(1024, kernel_initializer=init)(gen)
    gen = LeakyReLU(alpha=0.2)(gen)
    gen = Dense(28 * 28 * 1, kernel_initializer=init)(gen)
    out_layer = Activation('sigmoid')(gen)
    out_layer = Reshape((28, 28, 1))(out_layer)
    model = Model(in_lat, out_layer)
    return model

In [ ]:
generator = define_generator(100)


In [ ]:
images, y=generate_fake_samples(generator,100,64)

In [ ]:
images.shape
images1=images.reshape(64,28,28)
images1.shape

In [ ]:
plt.imshow(images1[63],'gray')

In [ ]:
def define_discriminator(in_shape=(28, 28, 1)):
    in_image = Input(shape=in_shape)
    fe = Flatten()(in_image)
    fe = Dense(1024)(fe)
    fe = LeakyReLU(alpha=0.2)(fe)
    fe = Dropout(0.3)(fe)
    fe = Dense(512)(fe)
    fe = LeakyReLU(alpha=0.2)(fe)
    fe = Dropout(0.3)(fe)
    fe = Dense(256)(fe)
    fe = LeakyReLU(alpha=0.2)(fe)
    fe = Dropout(0.3)(fe)
    out = Dense(1, activation='sigmoid')(fe)
    model = Model(in_image, out)
    opt = Adam(lr=0.0002, beta_1=0.5) 
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

In [ ]:
discriminator = define_discriminator()

In [ ]:
def define_gan(g_model, d_model):
    d_model.trainable = False #disabled the discriminator 
    gan_output = d_model(g_model.output)#in this we just use the generator 
    model = Model(g_model.input, gan_output)
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

In [ ]:
gan_model=define_gan(generator,discriminator)

In [ ]:
gan_model = define_gan(generator, discriminator)

In [ ]:
 for i in range(200):
   X_real, y_real = generate_real_samples(X_train1, 60000)
   d_loss_r, d_acc_r = discriminator.train_on_batch(X_real, y_real)
   X_fake, y_fake = generate_fake_samples(generator, 100, 60000)
   d_loss_f, d_acc_f = discriminator.train_on_batch(X_fake, y_fake)
   z_input = generate_latent_points(100, 60000) 
   y_gan = ones((60000, 1)) 
   g_loss, g_acc = gan_model.train_on_batch(z_input, y_gan)
   print('>%d, dr[%.3f,%.3f], df[%.3f,%.3f], g[%.3f,%.3f]' 
         % (i+1, d_loss_r,d_acc_r, d_loss_f,d_acc_f, g_loss,g_acc))

In [ ]:

images2, y=generate_fake_samples(generator,100,20)

In [ ]:
images2.shape
images2=images2.reshape(20,28,28)
images2.shape


In [ ]:
plt.imshow(images2[4],'gray')

In [ ]:
def train(g_model, d_model, gan_model, X_train, latent_dim, n_epochs=2, n_batch=128):
    bat_per_epo = int(X_train.shape[0] / n_batch)
    n_steps = bat_per_epo * n_epochs
    for i in range(n_steps):
        X_real, y_real = generate_real_samples(X_train, n_batch)
        d_loss_r, d_acc_r = d_model.train_on_batch(X_real, y_real)
        X_fake, y_fake = generate_fake_samples(g_model, latent_dim, n_batch)
        d_loss_f, d_acc_f = d_model.train_on_batch(X_fake, y_fake)
        z_input = generate_latent_points(latent_dim, n_batch) 
        y_gan = ones((n_batch, 1)) 
        g_loss, g_acc = gan_model.train_on_batch(z_input, y_gan)
        print('>%d, dr[%.3f,%.3f], df[%.3f,%.3f], g[%.3f,%.3f]' % (i+1, d_loss_r,d_acc_r, d_loss_f,d_acc_f, g_loss,g_acc))
        if (i+1) % (bat_per_epo * 1) == 0:
            summarize_performance(i, g_model, latent_dim)

In [ ]:
def summarize_performance(step, g_model, latent_dim, n_samples=100):
    X, _ = generate_fake_samples(g_model, latent_dim, n_samples)
    X = (X + 1) / 2.0
    for i in range(100):
        plt.subplot(10, 10, 1 + i)
        plt.axis('off')
        plt.imshow(X[i, :, :, 0], cmap='gray_r')
    filename2 = 'model_%04d.h5' % (step+1)
    g_model.save('/content/drive/My Drive/Colab Notebooks/'+ filename2)
    print('>Saved: %s' % (filename2))

In [ ]:
def save_plot(examples, n_examples):
    for i in range(n_examples):
        plt.subplot(sqrt(n_examples), sqrt(n_examples), 1 + i)
        plt.axis('off')
        plt.imshow(examples[i, :, :, 0], cmap='gray_r')
    plt.show()

In [ ]:
latent_dim = 100
train(generator, discriminator, gan_model, X_train1, 100, n_epochs=2, n_batch=128)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
import os 
os.getcwd()
os.chdir('/content/drive/My Drive/Colab Notebooks/')

In [ ]:
model = load_model('model_0936.h5')
latent_dim = 100
n_examples = 100
latent_points = generate_latent_points(latent_dim, n_examples)
X  = model.predict(latent_points)
X = (X + 1) / 2.0
save_plot(X, n_examples)